In [2]:
import tensorflow_federated as tff
tff.__version__

'0.87.0'

In [3]:
!python --version

Python 3.11.12


In [4]:
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt 
from pycocotools.coco import COCO 
import os
import urllib.request

In [ ]:
# Download the annotations file for COCO dataset
# ANNOTATIONS_FILE = "instances_val2017.json"
# ANNOTATIONS_URL = f"http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
# if not os.path.exists(ANNOTATIONS_FILE):
#     urllib.request.urlretrieve(ANNOTATIONS_URL, "coco_annotations.zip")    !unzip -q coco_annotations.zip -d .

In [65]:
# Load COCO dataset annotations
ANNOTATIONS_FILE = "instances_val2017.json"
coco = COCO(f'annotations/{ANNOTATIONS_FILE}')

# Function to load and preprocess images
def load_image(image_id):
    image_info = coco.loadImgs(image_id)[0]
    image_path = f"http://images.cocodataset.org/val2017/{image_info['file_name']}"

    # Load and preprocess image
    image = tf.io.decode_jpeg(tf.io.read_file(tf.keras.utils.get_file(image_info['file_name'], image_path)))
    image = tf.image.resize(image, (128, 128))
    image = image / 255.0

    # Get label (just the first annotation category_id for simplicity)
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)
    if not anns:
        label = 0  # default/fallback class
    else:
        label = anns[0]['category_id']  # use first category_id

    # Map COCO category_id (not consecutive) to a smaller range (0-9)
    label = label % 10  # reduce to 10 classes just for demo
    label = tf.one_hot(label, 10)  # one-hot encode

    return image, label

def make_client_data(image_ids):
    data = [load_image(img_id) for img_id in image_ids]
    images, labels = zip(*data)  # Unzip into separate lists
    return tf.data.Dataset.from_tensor_slices((list(images), list(labels))).batch(1)


federated_train_data = [make_client_data([img_id]) for img_id in image_ids]



loading annotations into memory...
Done (t=0.35s)
creating index...
index created!


In [ ]:
def create_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # Assuming 10 classes
    ])
    return model
# Convert to TFF model
def model_fn():
    keras_model = create_cnn_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )
    return tff_model

In [ ]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(0.001),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(1.0)
    )


AttributeError: 'function' object has no attribute 'initialize'

In [ ]:
NUM_ROUNDS = 5
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f'Round {round_num}, Metrics: {metrics}')

In [ ]:
evaluation_model = create_cnn_model()
state.model.assign_weights_to(evaluation_model)
# Load new test images
test_data = tf.data.Dataset.from_tensor_slices([load_image(img_id) for img_id in image_ids[5:10]]).batch(1)
loss, accuracy = evaluation_model.evaluate(test_data)
print(f'Test Accuracy: {accuracy:.4f}')